# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record



In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mean
import json
import matplotlib.pyplot as plt

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio = portfolio[portfolio.columns[::-1]]


In [3]:
portfolio

,id,offer_type,duration,difficulty,channels,reward
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,7,10,"[email, mobile, social]",10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,5,10,"[web, email, mobile, social]",10
2,3f207df678b143eea3cee63160fa8bed,informational,4,0,"[web, email, mobile]",0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,7,5,"[web, email, mobile]",5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,10,20,"[web, email]",5
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,7,"[web, email, mobile, social]",3
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,10,"[web, email, mobile, social]",2
7,5a8bc65990b245e5a138643cd4eb9837,informational,3,0,"[email, mobile, social]",0
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,"[web, email, mobile, social]",5
9,2906b810c7d4411798c6938adc9daaa5,discount,7,10,"[web, email, mobile]",2


In [4]:
profile

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [5]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [90]:
# modify the transcript data by unpacking the dictionary consisting column:
def df_unpack(df):
    """
    INPUT: 
        df (dataframe): pandas dataframe containing person, event, value, and time data
        
    OUTPUT:
        df_unpacked (dataframe): the same input data frame with the value column unpacked to 'tag' and 
        'id_amount' columns
    """
    df_unpacked = df.copy()
    
    # upacking the dictionaries in the column 'value'
    Idx=df_unpacked.set_index(['person',
                                      'event', 
                                      'time']).value.apply(pd.Series).stack().index
    df_unpacked = pd.DataFrame(df_unpacked.
                                      set_index(['person','event', 'time']).
                                      value.apply(pd.Series).stack().
                                      values.tolist(),index=Idx).reset_index()

    # reordering and renaming the columns:
    df_unpacked = df_unpacked[['person', 'event','level_3', 0, 'time']]
    df_unpacked.rename(columns={ "level_3": "tag", 0: "id_amount"}, inplace=True)
    df_unpacked['tag'] = df_unpacked['tag'].str.replace(' ', '_')
    
    return df_unpacked

In [91]:
# use our df_unpack function to unpack transcript data:
transcript_no_dict = df_unpack(transcript)

In [99]:
# define a function that takes the unpacked transcipt df and returns rewards and transaction dataframes:
def rewards_transaction(transcript_no_dict):
    """
    INPUTS:
        transcript_no_dict (dataframe): The unpacked df with person, event, tag, id_amount and time columns
        
    OUTPUTS: 
        transaction_df_person (dataframe): filtered transaction data for each person from the input dataframe
        reward_df_person (dataframe): filtered reward data for each person from the input dataframe
    """
    # creating a separate transaction data for calculating sum of transactions for each unique user:
    transcript_transactions = transcript_no_dict[transcript_no_dict['event']=='transaction']
    transcript_transactions.loc[:, 'id_amount'] = transcript_transactions.loc[:, 'id_amount'].astype(float)
    transcript_transactions.rename(columns={ "id_amount": "transaction_amount"}, inplace=True)
    transaction_df_person = (transcript_transactions.groupby('person')[['transaction_amount']].sum())
    transaction_df_person.reset_index(inplace=True)
    
    # creating reward dataset group by person id:
    reward_df = transcript_no_dict[transcript_no_dict['tag']=='reward']
    reward_df.loc[:,'id_amount'] = reward_df.loc[:,'id_amount'].astype(int)
    reward_df.rename(columns={ "id_amount": "reward_amount"}, inplace=True)
    reward_df_person = pd.DataFrame(reward_df.groupby('person')[['reward_amount']].sum())
    reward_df_person.reset_index(inplace=True)
    
    return transaction_df_person, reward_df_person

In [101]:
# use our rewards_transaction() function to create our dataframes:
transaction_df_person, reward_df_person = rewards_transaction(transcript_no_dict)

In [104]:
# defining a function to return grouped datasets for each person including details about their engagements 
# with the offers:
def person_offer_id_df(transcript_no_dict):
    """
    INPUT:
        transcript_no_dict (dataframe): The unpacked df with person, event, tag, id_amount and time columns
        
    OUTPUT:
        transcript_offer_id (dataframe): offer id data set grouped by person, event and status
        person_offer (dataframe): data set including number of each event for each person
    """
    # creating a dataframe containing all the affer related events:
    transcript_offer = transcript_no_dict[transcript_no_dict['event']!='transaction']

    # creating offer id data set grouped by person, event and status:
    transcript_offer_id = transcript_offer[transcript_offer['tag']=='offer_id']
    transcript_offer_id.rename(columns={ "id_amount": "offer_id"}, inplace=True)

    # creating person-offer dataframe:
    person_offer = transcript_offer_id.groupby(['person', 'event'])['offer_id'].count().unstack()
    person_offer.reset_index(inplace=True)
    
    return transcript_offer_id, person_offer

In [106]:
transcript_offer_id, person_offer = person_offer_id_df(transcript_no_dict)

In [107]:
# defining a fuction that finds each persons average view time and action time:
def get_times(person_id, df=transcript_offer_id):
    """
    INPUT: 
        person_id (string): the person id for whom we want to extract the timings for
        df (dataframe): a data frame with person , event, and offer_id data
        
    OUTPUT: 
        view_time_mean (int): the average time needed for the person to view the offer since it was received
        action_time_mean (int): the average time needed for the person to act on the viewed offer
    """
    p1 = df[df['person']== person_id]
    
    view_time_list = []
    action_time_list = []

    for i in list(p1['offer_id'].unique()):
        event_offer = list(p1[p1['offer_id']==i]['event'])
                
        if 'offer viewed' in event_offer:
            off_rec_time = p1[(p1['offer_id']==i) & 
                          (p1['event']=='offer received')]['time'].values[0]
            off_vie_time = p1[(p1['offer_id']==i) & 
                          (p1['event']=='offer viewed')]['time'].values[0]
            
            vt = off_vie_time - off_rec_time
        
        else:
            vt = float('NaN')
        
        if 'offer viewed' in event_offer and 'offer completed' in event_offer:
            off_com_time = p1[(p1['offer_id']==i) & 
                              (p1['event']=='offer completed')]['time'].values[0]
            
            at = off_com_time - off_vie_time
        else:
            at = float('NaN')
        
        # if view time is NaN, we'll know it does not exist, so we will not add it to our list
        if math.isnan(vt)==False and vt >= 0:
            view_time_list.append(vt)
        else:
            pass

        # negative action time means that some of offer completions were done before viewing the offer
        # so we will not add them to our list
        if math.isnan(at)==False and at >= 0:
            action_time_list.append(at)
        else:
            pass
    try:
        view_time_mean = mean(view_time_list)
    except:
        view_time_mean = float('NaN')
    
    try:
        action_time_mean = mean(action_time_list)
    except:
        action_time_mean = float('NaN')
        
    
    return view_time_mean, action_time_mean

In [108]:
def view_action_time_add(person_offer):
    """
    INPUT: 
        person_offer (dataframe): data set including number of each event for each person
        
    OUTPUT:
        person_offer (dataframe): input data frame including added view_time and action_time columns
    """
    # creating the view time and action time columns using get_times() function:
    view_times = []
    action_times = []
    for i in person_offer['person']:
        view_times.append(get_times(i)[0])
        action_times.append(get_times(i)[1])
    # adding the new lists to our person_offer df:
    person_offer['view_time'] = view_times
    person_offer['action_time'] = action_times
    
    return person_offer

In [109]:
person_offer = view_action_time_add(person_offer)

In [111]:
# merging all 3 user-based dataframes to have a more detailed data on 
# each persons interaction with the offers:
merged_df = pd.merge(reward_df_person, transaction_df_person, how='right', on= 'person')
person_merged = pd.merge(merged_df, person_offer, how='right', on='person')
person_merged

,person,reward_amount,transaction_amount,offer completed,offer received,offer viewed,view_time,action_time
0,0009655768c64bdeb2e877511632db8f,9.0,127.60,3.0,5.0,4.0,36.0,NaN
1,00116118485d4dfda04fdbaba9a87b5c,NaN,4.09,NaN,2.0,2.0,48.0,NaN
2,0011e0d4e6b944f998e987f904e8c1e5,13.0,79.46,3.0,5.0,5.0,15.0,90.0
3,0020c2b971eb4e9188eac86d93036a77,14.0,196.86,3.0,5.0,3.0,62.0,63.0
4,0020ccbbb6d84e358d3414a3ff76cffd,13.0,154.05,3.0,4.0,4.0,22.0,34.0
...,...,...,...,...,...,...,...,...
16989,fff3ba4757bd42088c044ca26d73817a,9.0,580.98,3.0,6.0,3.0,84.0,0.0
16990,fff7576017104bcc8677a8d63322b5e1,9.0,29.94,3.0,5.0,4.0,14.0,168.0
16991,fff8957ea8b240a6b5e634b6ee8eafcf,NaN,12.15,NaN,3.0,2.0,54.0,NaN
16992,fffad4f4828548d1b5583907f2e9906b,15.0,88.83,3.0,4.0,4.0,32.0,30.0


### create a dataframe with all the data from profile df and person merged df and prepare for modeling

In [54]:
# first start by checking why there are 6 missing user id s in the merged df!
missing_ids = list(np.setdiff1d(np.array(profile['id']), 
                                np.array(person_merged['person'])))
transcript_no_dict[transcript_no_dict['person'].isin(missing_ids)]['event'].unique()

array(['transaction'], dtype=object)

We clearly see that the persons who are not in the finale merged df, all have done transactions without getting an offer, so it's better to leave it as it is.

In [57]:
# Now we sort and rename the profile df to prepare it for the merge:
sorted_columns = ['id','gender', 'age','became_member_on', 'income']
profile = profile[sorted_columns]
profile.rename(columns={ "id": "person"}, inplace=True)

# merge and reorder the datasets:
profile_merged = pd.merge(person_merged, profile, how='right', on= 'person')
profile_merged = profile_merged[['person', 'gender','age', 'became_member_on', 
                                 'income', 'reward_amount', 'transaction_amount', 
                                 'view_time', 'action_time', 
                                 'offer completed']]

In [83]:
profile_merged

,person,gender,age,became_member_on,income,reward_amount,transaction_amount,view_time,action_time,offer completed
0,68be06ca386d4c31939f3a4f0e3dd783,None,118,20170212,NaN,5.0,20.40,15.0,96.0,2.0
1,0610b486422d4921ae7d2bf64640c50b,F,55,20170715,112000.0,5.0,77.01,NaN,NaN,1.0
2,38fe809add3b4fcf9315a9694bb96ff5,None,118,20180712,NaN,NaN,14.30,45.0,NaN,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,F,75,20170509,100000.0,20.0,159.27,33.0,114.0,3.0
4,a03223e636434f42ac4c3df47e8bac43,None,118,20170804,NaN,NaN,4.65,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,F,45,20180604,54000.0,NaN,20.03,44.0,NaN,NaN
16996,2cb4f97358b841b9a9773a7aa05a9d77,M,61,20180713,72000.0,5.0,25.97,42.0,NaN,1.0
16997,01d26f638c274aa0b965d24cefe3183f,M,49,20170126,73000.0,NaN,39.74,60.0,NaN,NaN
16998,9dc1421481194dcd9400aec7c9ae6366,F,83,20160307,50000.0,25.0,189.67,54.0,18.0,3.0


Now we should filter out the rows with existing 'offer completed' values and NaN 'action time' value. These are users who completed an offer before viewing it (with reference to our get_times() function in which we put NaN for all negative time differences between view time and action time) so they would do it regardless of viewing the offer.

In [86]:
non_engaged = list((profile_merged[(profile_merged['offer completed'].isnull()==False) & 
              (profile_merged['action_time'].
               isnull()==True)])['person'])

In [89]:
# droping the rows containing non-engaged users
profile_merged_clean = profile_merged[~profile_merged['person'].isin(non_engaged)]

In [112]:
profile_merged_clean

,person,gender,age,became_member_on,income,reward_amount,transaction_amount,view_time,action_time,offer completed
0,68be06ca386d4c31939f3a4f0e3dd783,None,118,20170212,NaN,5.0,20.40,15.0,96.0,2.0
2,38fe809add3b4fcf9315a9694bb96ff5,None,118,20180712,NaN,NaN,14.30,45.0,NaN,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,F,75,20170509,100000.0,20.0,159.27,33.0,114.0,3.0
4,a03223e636434f42ac4c3df47e8bac43,None,118,20170804,NaN,NaN,4.65,3.0,NaN,NaN
5,e2127556f4f64592b11af22de27a7932,M,68,20180426,70000.0,7.0,57.73,16.0,51.0,2.0
...,...,...,...,...,...,...,...,...,...,...
16994,c02b10e8752c4d8e9b73f918558531f7,None,118,20151211,NaN,NaN,12.78,25.0,NaN,NaN
16995,6d5f3a774f3d4714ab0c092238f3a1d7,F,45,20180604,54000.0,NaN,20.03,44.0,NaN,NaN
16997,01d26f638c274aa0b965d24cefe3183f,M,49,20170126,73000.0,NaN,39.74,60.0,NaN,NaN
16998,9dc1421481194dcd9400aec7c9ae6366,F,83,20160307,50000.0,25.0,189.67,54.0,18.0,3.0


In [ ]:
# after that we should create a different df based on offers
# to identify which groups of people are most responsive to each type of offer, 
# and how best to present each type of offer

In [312]:
# creating offer-person dataframe:
offer_person = transcript_offer_id.groupby(['offer_id', 'event'])['person'].count().unstack()
offer_person.reset_index(inplace=True)

In [313]:
offer_person

event,offer_id,offer completed,offer received,offer viewed
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,3420.0,7668.0,2663.0
1,2298d6c36e964ae4a3e7e9706d1fb8c2,5156.0,7646.0,7337.0
2,2906b810c7d4411798c6938adc9daaa5,4017.0,7632.0,4118.0
3,3f207df678b143eea3cee63160fa8bed,NaN,7617.0,4144.0
4,4d5c57ea9a6940dd891ad53e9dbe8da0,3331.0,7593.0,7298.0
5,5a8bc65990b245e5a138643cd4eb9837,NaN,7618.0,6687.0
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,4354.0,7677.0,4171.0
7,ae264e3637204a6fb9bb56bc8210ddfd,3688.0,7658.0,6716.0
8,f19421c1d4aa40978ebb69ca19b0e20d,4296.0,7571.0,7264.0
9,fafdcd668e3743c1bb461111dcafc2a4,5317.0,7597.0,7327.0
